# Text Representation

Please, note that this notebook is intended to be run in Google Colab.

In [1]:
train_file ='train.pkl'
test_file = 'test.pkl'
meta_file = 'meta_All_Beauty.json.gz'

# Exercise 1

Load the [metadata file](https://nijianmo.github.io/amazon/index.html) and discard any item that was not rated by our subset of users (nor in training or test sets). Apply preprocessing (stemming and stopwords removal) to clean up the text from the "title". Report the vocabulary size before and after the preprocessing.

In [2]:
import gzip
import os
import sys
sys.path.append('../')
import pickle
import pandas as pd
import json
import numpy as np

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


train = pd.read_pickle("train.pkl")
test = pd.read_pickle("test.pkl")


meta = getDF(meta_file)
print("Before dropping dups:", len(meta))
meta = meta.drop_duplicates(["asin"], keep="last")
print("After:", len(meta))


Before dropping dups: 32892
After: 32488


In [3]:
allowed_asins = np.unique(np.array([*train.asin.to_list(), *test.asin.to_list()]))
meta = meta[meta["asin"].isin(allowed_asins)].reset_index(drop=True)

len(allowed_asins), len(meta)

(84, 84)

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ayaya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayaya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from bs4 import BeautifulSoup
import re
# https://stackoverflow.com/questions/45670532/stemming-words-with-nltk-python 


def cleanWords(x, stemmer=PorterStemmer()):
    txt = BeautifulSoup(x).get_text()              #remove html
    letters = re.sub("[^a-z]", " ", txt.lower())   #lowercase and only letters
    words   = word_tokenize(letters)               #tokenize
    stops = set(stopwords.words("english"))        #define stopwords 
    meaningful_words = [w for w in words if not w in stops] 

    return ' '.join([stemmer.stem(w) for w in meaningful_words]) #combine

df = meta.copy()
df["words"] = df["title"].apply(lambda x: cleanWords(x))

df["words"]

0                aqua velva shave classic ice blue ounc
1                citr shine moistur burst shampoo fl oz
2                                   nar blush taj mahal
3        avalon organ wrinkl therapi coq cleans milk oz
4                          zum zum bar anis lavend ounc
                            ...                        
79                     ultim bodi lotion michael kor oz
80                     dolc gabbana compact parfum ounc
81    colgat kid maximum caviti protect pump toothpa...
82    bali secret natur deodor organ vegan women men...
83                          essi gel coutur nail polish
Name: words, Length: 84, dtype: object

In [21]:
# unique words in vocab (includes 1-letter stems)
len(set(' '.join(df["words"]).split(' ')))

401

# Exercise 2

Representation in vector spaces.

## 2.1

Represent all the products from Exercise 1 in a TF-IDF space. Interpret the meaning of the TF-IDF matrix dimensions.

Tip: You may use the library [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) 

In [25]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df["words"].tolist())
tfidf_vectorizer.get_feature_names_out().shape

(394,)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(df["words"])
tfidf = tfidf_vectorizer.fit_transform(df["words"])
arr = tfidf.toarray()

arr.shape, arr, tfidf_vectorizer.get_feature_names_out().shape

((84, 394),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (394,))

In [70]:
dfwrite = pd.concat([df["asin"],pd.DataFrame(arr)],axis=1)
dfwrite.to_pickle(p+"asin_tfidf.pkl")

In [58]:
# with open(p+"tfidf", 'wb') as f:
#     np.save(f, arr)
# testset.to_pickle("testset.pkl")
# arr.to_pickle("tfidf.pkl")
# df.to_pickle("text_representation.pkl")

In [10]:
ids = df.where(df["asin"].isin(["B000FI4S1E", "B000LIBUBY", "B000W0C07Y"])).dropna()
ids#, arr[ids]

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,words,tfidf
6,[],,[Fruits &amp; Passion SOLSTIS Refreshing Showe...,,Fruits &amp; Passion Blue Refreshing Shower Ge...,[],,Fruits & Passion,[],"2,539,624 in Beauty & Personal Care (",...,"{'Shipping Weight:': '8 ounces', 'ASIN: ': 'B0...",All Beauty,,,,B000FI4S1E,[],[],fruit passion blue refresh shower gel 6 7 fl oz,"[0.0, 0.20874267904919294, 0.0, 0.0, 0.0, 0.0,..."
10,[],,[<li>A brilliant effervescent fragrance for yo...,,"Fresh Eau de Parfum, Sugar Lemon, 3.4 oz",[],,Fresh,[],"572,901 in Beauty & Personal Care (",...,"{' Item Weight: ': '3.36 ounces', 'Sh...",All Beauty,,,$99.25,B000LIBUBY,[],[],fresh eau de parfum sugar lemon 3 4 oz,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17,[],,"[, For Women]",,Sex In The City Kiss by Instyle Parfums Eau De...,[],,Instyle Parfums,[],"616,259 in Beauty & Personal Care (",...,"{' Item Weight: ': '0.8 ounces', 'Shi...",All Beauty,,,,B000W0C07Y,[],[],sex citi kiss instyl parfum eau de parfum spra...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [30]:
import texthero as hero
df['tfidf'] = hero.tfidf(df['title'])
df["tfidf"] = df["tfidf"].apply(lambda x: np.array(x))
subset = df.loc[ids.index][["asin", "tfidf"]]
subset

,asin,tfidf
6,B000FI4S1E,"[0.0, 0.20874267904919294, 0.0, 0.0, 0.0, 0.0,..."
10,B000LIBUBY,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17,B000W0C07Y,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## 2.2

Compute and the cosine similarity between products with asin 'B000FI4S1E', 'B000LIBUBY' and 'B000W0C07Y'. Take a look at their features to see whether results make sense with their characteristics. 

In [49]:
# cos_sim = (a @ b.T) / (norm(a)*norm(b))
cosine_similarity(np.array(subset.tfidf.to_list()))
cosine_similarity(arr[[6,10,17]])

array([[1.        , 0.0348894 , 0.02611649],
       [0.0348894 , 1.        , 0.44104404],
       [0.02611649, 0.44104404, 1.        ]])

# Exercise 3

Representation in vector spaces with contextual Word Embeddings.

## 3.1.

Represent all the products from Exercise 1 in a vector space using embeddings from a pre-trained BERT model. The final embedding of a product should be the average of the word embeddings from all the words in the 'title'. What is the vocabulary size of the model? What are the dimensions of the last hidden state?

Tip: you may install the transformers library and use their pretrained [BERT model uncased](https://huggingface.co/bert-base-uncased).

In [ ]:
# LOAD TRANSFORMER
"""
If you plan on using a pretrained model, it’s important to use the associated 
pretrained tokenizer: it will split the text you give it in tokens the same way
for the pretraining corpus, and it will use the same correspondence
token to index (that we usually call a vocab) as during pretraining.
"""

# % pip install transformers
import torch
import transformers
assert transformers.__version__ > '4.0.0'

from transformers import BertModel, BertTokenizerFast

# set-up environment
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


modelname = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(modelname)
model = BertModel.from_pretrained(modelname).to(DEVICE)

# Print out the vocabulary size
# <YOUR CODE HERE>

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [ ]:
# REPRESENT PRODUCTS IN A VECTOR SPACE


def batch_encoding(sentences):
    # Since we're using padding, we need to provide the attention masks to our
    # model. Otherwise it doesn't know which tokens it should not attend to. 
    inputs = # <YOUR CODE HERE>
    # print(inputs) # Look at the padding and attention_mask

    outputs = model(**inputs)

    last_hidden_states = # <YOUR CODE HERE>

    return inputs, last_hidden_states
  
encoded_inputs, title_last_hidden_states = batch_encoding( # <YOUR CODE HERE> )

"""
Note that the control token [CLS] has been added 
at the beginning of each sentence, and [SEP] at the end. 
"""

# Now, let's mask out the padding tokens and compute the embedding vector of each product

# <YOUR CODE HERE>

## 3.2.

Compute and the cosine similarity between products with asin 'B000FI4S1E', 'B000LIBUBY' and 'B000W0C07Y'.